In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import os
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
def variables_summary(var):
    with tf.name_scope("summaries"):
        # 平均值
        mean = tf.reduce_mean(var)
        # 给mean参数定义一个为mean的名字
        tf.summary.scalar("mean",mean)
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev",stddev) # 标准差
        tf.summary.scalar("max",tf.reduce_max(var)) # 最大值
        tf.summary.scalar("min",tf.reduce_min(var)) # 最小值
        tf.summary.histogram("histogram",var) # 直方图
sess = tf.Session()
# 图片数量
image_num = 3000
TSV = "projector/projector/metadata.tsv"
if os.path.exists(TSV):  # 如果文件存在
    # 删除文件，可使用以下两种方法。
    os.remove(TSV)  
# 产生metadata文件  就是将图片标签写入文件
with open(TSV,"w") as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')
# 将3000张图片打包
embedding_var = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name="embedding")
# 运行次数
max_steps = 1001
batch_size = 100
# 命名空间
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32,[None,784],name="x_input")
    y = tf.placeholder(tf.float32,[None,10],name="y_input")
# 显示图片
with tf.name_scope("input_reshape"):
    # -1 代表一个任意值 黑白图片 所以最后一个维度是1
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    # 显示10张图片
    tf.summary.image("input",image_shaped_input,10)
    
with tf.name_scope("layer"):
    with tf.name_scope("weights"):
        weights = tf.Variable(tf.zeros([784,10]),name="w")
        variables_summary(weights)
    with tf.name_scope("biases"):
        biases = tf.Variable(tf.zeros([10]),name="b")
        variables_summary(biases)
    with tf.name_scope("wx_plus_b"):
        out =tf.matmul(x,weights)+biases
    with tf.name_scope("softmax_out"):
        prediction = tf.nn.softmax(out)
        
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y-prediction))
    tf.summary.scalar("loss",loss)
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    


with tf.name_scope("acc"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar("accuracy",accuracy)
        
# 合并所有summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter("projector/projector",sess.graph)
# 用于保存网络模型
saver = tf.train.Saver()
# 配置项
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
embedding.metadata_path = TSV #'metadata.tsv'
embedding.sprite.image_path = "projector/data/mnist_10k_sprite.png" #'mnistdigits.png'
# 切分成每个28*28的小图
embedding.sprite.single_image_dim.extend([28,28])

# Say that you want to visualise the embeddings
projector.visualize_embeddings(projector_writer, config)
init = tf.global_variables_initializer()
sess.run(init)
for i in range(max_steps):
    batch_xs,batch_ys = mnist.train.next_batch(batch_size)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,"step%03d"%i)
    projector_writer.add_summary(summary,i)
    if i%100 == 0:
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Epoch:"+str(i)+",Accuracy"+str(acc))
saver.save(sess,"projector/projector/mymodel.ckpt",global_step=max_steps)
projector_writer.close()
sess.close()

Epoch:0,Accuracy0.3855
Epoch:100,Accuracy0.6356
Epoch:200,Accuracy0.6902
Epoch:300,Accuracy0.7577
Epoch:400,Accuracy0.7863
Epoch:500,Accuracy0.8224
Epoch:600,Accuracy0.8395
Epoch:700,Accuracy0.8505
Epoch:800,Accuracy0.8602
Epoch:900,Accuracy0.861
Epoch:1000,Accuracy0.8671


In [ ]:

# with tf.Session() as sess:
#     sess.run(init)
# #     logs 是存放路径，存放到当前目录下的logs目录中，没有该目录则自动生成
#     writer = tf.summary.FileWriter("logs/",sess.graph)
#     for epoch in range(51):
#         for batch in range(n_batch):
#                 batch_xs,batch_ys = mnist.train.next_batch(batch_size)
#                 summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
#         writer.add_summary(summary,epoch)
#         acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
#         print("Epoch:"+str(epoch)+",Accuracy"+str(acc))